In [1]:
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pymc3 as pm
import arviz as az
from sklearn.linear_model import LinearRegression
from typing import Dict, Optional, Tuple, Sequence, Iterator

In [2]:
from platform import python_version

print(python_version())

3.9.5


In [3]:
"""
Import data
"""
raw_data = pd.read_pickle("data.pkl")
raw_data[0:5]

,subject,exp,phasename,num_rep,num_trial,num_formed,num_uttered,prompt,in_or_out,date_run,time_run,block,pitches
0,20,PU,5,44,1,2637,70,HECK,O,20170127,11,3,0 134.355450 1 134.360625 2 134...
1,20,PU,5,36,1,2629,62,HECK,O,20170127,11,3,0 130.217779 1 130.179978 2 130...
2,20,PU,7,20,1,2713,146,HECK,I,20170127,11,3,0 132.486939 1 132.479573 2 132...
3,20,PU,6,6,1,2649,82,HECK,O,20170127,11,3,0 139.037150 1 139.055242 2 139...
4,20,PU,7,8,1,2701,134,HECK,I,20170127,11,3,0 130.927041 1 130.913059 2 130...


In [4]:
"""
Preprocessing
"""

# Create new data frame

# Remove "O" data
df = raw_data[raw_data.in_or_out == "O"]

# Remove first block
df = df[df.block != 1]

# Rename "phasename" to "phasenum"
df.rename(columns = {"phasename": "phasenum", "exp": "condition"}, inplace = True)

# Reset pd indexes
df.reset_index()

# Preview
df[0:5]

,subject,condition,phasenum,num_rep,num_trial,num_formed,num_uttered,prompt,in_or_out,date_run,time_run,block,pitches
0,20,PU,5,44,1,2637,70,HECK,O,20170127,11,3,0 134.355450 1 134.360625 2 134...
1,20,PU,5,36,1,2629,62,HECK,O,20170127,11,3,0 130.217779 1 130.179978 2 130...
3,20,PU,6,6,1,2649,82,HECK,O,20170127,11,3,0 139.037150 1 139.055242 2 139...
5,20,PU,1,1,1,2568,1,HECK,O,20170127,11,3,0 112.625662 1 112.649338 2 112...
7,20,PU,5,39,1,2632,65,HECK,O,20170127,11,3,0 139.263470 1 139.266789 2 139...


In [5]:
"""
Generate summary statistics for each trial
"""

agg: Dict[int, float] = {
    "count": [],
    "avg": [],
    "std": [],
    "minimum": [],
    "med": [],
    "maximum": [],
}
    
for pitches in df.pitches:
    (
        count,
        avg,
        std,
        minimum,
        _,
        med,
        _,
        maximum,
                ) = pitches.describe()
    agg["count"].append(count)
    agg["avg"].append(avg)
    agg["std"].append(std)
    agg["minimum"].append(minimum)
    agg["med"].append(med)
    agg["maximum"].append(maximum)
    
# Preview summary statistics
summary_stats = pd.DataFrame(agg)
summary_stats[0:5]

,count,avg,std,minimum,med,maximum
0,491.0,133.610873,1.544445,124.930762,133.915850,134.887327
1,531.0,128.870786,1.549235,124.541467,129.371750,132.296456
2,461.0,137.901782,0.786993,135.052405,137.730314,140.124944
3,141.0,115.535289,2.270588,112.625662,114.953949,121.897112
4,451.0,134.375365,2.289648,127.755729,133.575767,139.429389


In [6]:
"""
Relativize to F0 of start phase
"""

subjects = df.subject.unique()
conditions = df.condition.unique()

# Subset only start phase trials with the prompt "HECK"
selection = ((df.phasenum == 4) & (df.prompt == "HECK")).to_numpy()
trial_f0 = summary_stats.avg[selection]
trial_subject = df.subject[selection] 
trial_condition = df.condition[selection] 

# Get the baseline F0 for each subject
f0s = {}
for subject in subjects:
    for condition in conditions:
        selection = ((subject == trial_subject) & (condition == trial_condition)).to_numpy()
        trials = trial_f0[selection]
        f0 = trials.mean()
        f0s[(subject, condition)] = f0

# Subtract pitches from baseline F0 
corrected_pitches = []

# For each trial
for i, trial_pitches in enumerate(df.pitches):
        
    # Get the subject's f0
    subject = df.subject.to_numpy()[i]
    condition = df.condition.to_numpy()[i]
    f0 = f0s[(subject, condition)]
    
    # Subtract f0 from values
    trial_corrected_pitches = trial_pitches - f0
    
    # Take mean
    trial_corrected_pitches = trial_corrected_pitches.mean()
    
    # Save in new arrage
    corrected_pitches.append(trial_corrected_pitches)
    
cleaned_df = df.drop(columns = ['pitches', 'num_trial', 'in_or_out'])
cleaned_df['corrected_pitches'] = corrected_pitches
cleaned_df[0:5]

In [8]:
"""
Helper functions
"""

def subjects(df) -> Sequence[int]:
    return list(df.subject.unique())

PHASENUMS = [i for i in range(4, 8)]
CONDITIONS = ["PU", "PD", "FAE", "FIH"]
SUBJECTS = subjects(cleaned_df)

phasenames = {
    "2": "pract2",
    "3": "unk",
    "4": "start",
    "5": "ramp_up",
    "6": "stay",
    "7": "ramp_down",
    }
    
KeyType = Tuple[int, int, str]

def individual_keys() -> Iterator[KeyType]:
    for condition in CONDITIONS:
        for phasenum in PHASENUMS:
            for subject in SUBJECTS:
                key = (condition, phasenum, subject)
                yield key
                
def group_keys() -> Iterator[KeyType]:
    for condition in CONDITIONS:
        for phasenum in PHASENUMS:
            key = (condition, phasenum)
            yield key

def subset_data(
    df: pd.DataFrame,
    *,
    condition: Optional[str] = None,
    phasenum: Optional[int] = None,
    subject: Optional[int] = None,
) -> pd.DataFrame:
    
    if condition is not None:
        df = df[df.condition == condition]
    
    if phasenum is not None:
        df = df[df.phasenum == phasenum]
    
    if subject is not None:
        df = df[df.subject == subject]
    
    return df

In [9]:
"""
Scatterplots for reps and change in pitch
"""

def plot_scatter(df: pd.DataFrame, phasenames: dict):
    # Get phasename
    phasename = phasenames[str(phasenum)]

    # Plot
    x = df.num_rep
    y = df.corrected_pitches
    colors = df.subject
    plt.scatter(x, y, c = colors)
    plt.title(phasename + " phase for " + condition)
    plt.xlabel("Rep")
    plt.ylabel("Change in F0")
    plt.ylim([-100, 100])
    plt.savefig("figs/" + phasename + "_" + condition + ".png")
    plt.close()

for key in group_keys():
    condition, phasenum = key
    plot_df = subset_data(cleaned_df, condition = condition, phasenum = phasenum)
    plot_scatter(plot_df, phasenames)

In [24]:
"""
Linear regression between rep and change in pitch over all subjects
"""

agg_bayes = {
    "condition": [],
    "phasenum": [],
    "param": [],
    "avg": [],
    "std": [],
    "hdi_upper": [],
    "hdi_lower": [],
}

for key in group_keys():
    print("Fitting linear model for:", key)
    
    # Get data for model
    condition, phasenum = key
    test_df = subset_data(cleaned_df, condition = condition, phasenum = phasenum)
    x = test_df.num_rep.to_numpy().reshape(-1, 1)
    x = x.flatten()
    y = test_df.corrected_pitches.to_numpy()
    
    # Model
    with pm.Model() as model:
        # Define priors
        a = pm.Normal('slope', 0, 16)
        b = pm.Normal('intercept', 0, 16)
        s = pm.Exponential('error', 1)

        # Predictions
        obs = pm.Normal('observation', a * x + b, s, observed = y)

        # Use MCMC to sample
        trace = pm.sample(1000, tune=1000, cores=1)

        # Get summary statistics for posterior
        posteriors = az.summary(trace, round_to = 2, hdi_prob = 0.95)
        for i, param in enumerate(["b", "a"]):
            avg = posteriors.iloc[i, 0]
            std = posteriors.iloc[i, 1]
            hdi_lower = posteriors.iloc[i, 2]
            hdi_upper = posteriors.iloc[i, 3]
            agg_bayes["condition"].append(condition)
            agg_bayes["phasenum"].append(phasenum)
            agg_bayes["param"].append(param)
            agg_bayes["avg"].append(avg)
            agg_bayes["std"].append(std)
            agg_bayes["hdi_lower"].append(hdi_lower)
            agg_bayes["hdi_upper"].append(hdi_upper)
        
agg_bayes = pd.DataFrame(agg_bayes)

# add error term to quantify change in variability!!

Fitting linear model for: ('PU', 4)


<ipython-input-24-aa9ed8658a8e>:36: FutureWarning: In v4.0, pm.sample will return an `arviz.InferenceData` object instead of a `MultiTrace` by default. You can pass return_inferencedata=True or return_inferencedata=False to be safe and silence this warning.
  trace = pm.sample(1000, tune=1000, cores=1)
Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Sequential sampling (2 chains in 1 job)
NUTS: [error, intercept, slope]


Sampling 2 chains for 1_000 tune and 1_000 draw iterations (2_000 + 2_000 draws total) took 4 seconds.
The acceptance probability does not match the target. It is 0.8810414895868006, but should be close to 0.8. Try to increase the number of tuning steps.


Fitting linear model for: ('PU', 5)


/opt/anaconda3/envs/fa/lib/python3.9/site-packages/pymc3/model.py:1755: ImputationWarning: Data in observation contains missing values and will be automatically imputed from the sampling distribution.
  warnings.warn(impute_message, ImputationWarning)
<ipython-input-24-aa9ed8658a8e>:36: FutureWarning: In v4.0, pm.sample will return an `arviz.InferenceData` object instead of a `MultiTrace` by default. You can pass return_inferencedata=True or return_inferencedata=False to be safe and silence this warning.
  trace = pm.sample(1000, tune=1000, cores=1)
Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Sequential sampling (2 chains in 1 job)
NUTS: [observation_missing, error, intercept, slope]


...

<ipython-input-24-aa9ed8658a8e>:36: FutureWarning: In v4.0, pm.sample will return an `arviz.InferenceData` object instead of a `MultiTrace` by default. You can pass return_inferencedata=True or return_inferencedata=False to be safe and silence this warning.
  trace = pm.sample(1000, tune=1000, cores=1)
Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Sequential sampling (2 chains in 1 job)
NUTS: [error, intercept, slope]


Sampling 2 chains for 1_000 tune and 1_000 draw iterations (2_000 + 2_000 draws total) took 5 seconds.


Fitting linear model for: ('FIH', 7)


/opt/anaconda3/envs/fa/lib/python3.9/site-packages/pymc3/model.py:1755: ImputationWarning: Data in observation contains missing values and will be automatically imputed from the sampling distribution.
  warnings.warn(impute_message, ImputationWarning)
<ipython-input-24-aa9ed8658a8e>:36: FutureWarning: In v4.0, pm.sample will return an `arviz.InferenceData` object instead of a `MultiTrace` by default. You can pass return_inferencedata=True or return_inferencedata=False to be safe and silence this warning.
  trace = pm.sample(1000, tune=1000, cores=1)
Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Sequential sampling (2 chains in 1 job)
NUTS: [observation_missing, error, intercept, slope]


Sampling 2 chains for 1_000 tune and 1_000 draw iterations (2_000 + 2_000 draws total) took 7 seconds.


In [27]:
agg_bayes[0:50]
agg_bayes.to_csv("group_lin_reg.csv", index = False)

In [18]:
lin_reg_summary.loc[["slope"], ["mean"]]
lin_reg_summary.iloc[1, 1]

0.8

In [17]:
lin_reg_summary

,mean,sd,hdi_2.5%,hdi_97.5%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
slope,-0.02,0.05,-0.12,0.09,0.00,0.00,1149.32,1115.30,1.01
intercept,0.27,0.80,-1.21,1.84,0.02,0.02,1160.19,1014.02,1.00
observation_missing[0],0.19,10.53,-19.91,20.68,0.23,0.25,2010.58,1499.18,1.00
error,10.46,0.26,9.93,10.97,0.01,0.00,1791.54,1307.35,1.00


In [15]:
for i, letter in enumerate(["a", "b"]):
    print(i, letter)

0 a
1 b


In [29]:
raw_data.phasename.unique()

array([5, 7, 6, 1, 4, 2, 3])

AttributeError: 'LinearRegression' object has no attribute 'summary_'

In [79]:
"""
Linear regression between rep and change in pitch for individual subjects
"""

subjects = cleaned_df.subject.unique()

# Make data frame to fill

def lin_reg(df: pd.DataFrame, phasenames: dict) -> Tuple[float, float]:
    # Get phasename
    phasename = phasenames[str(phasenum)]
    
    # Lin reg
    x = df.num_rep.to_numpy().reshape(-1, 1)
    y = df.corrected_pitches
    y.fillna(y.mean(), inplace = True)
    y = y.to_numpy().reshape(-1, 1)
    if x.shape[0] == 0 and y.shape[0] == 0:
        print("No data for", (condition, phasenum, subject))
        return None
    ln = LinearRegression().fit(x, y)
    a = ln.intercept_[0]
    b = ln.coef_[0][0]
    return a, b

subject_regressions = []

for key in individual_keys():
    condition, phasenum, subject = key
    lin_reg_df = subset_data(cleaned_df, condition = condition, phasenum = phasenum, subject = subject)
    regression = lin_reg(lin_reg_df, phasenames)
    if regression is None:
        continue
    a, b = regression
    subject_regressions.append((phasenum, condition, subject, a, b))

No data for ('FIH', 7, 16)


In [11]:
ValueType = pd.DataFrame
AggBayesType = Dict[KeyType, ValueType]
agg_bayes: AggBayesType = {}

for key in group_keys():
    print("Fitting linear model for:", key)
    
    # Get data for model
    condition, phasenum = key
    test_df = subset_data(cleaned_df, condition = condition, phasenum = phasenum)
    x = test_df.num_rep.to_numpy().reshape(-1, 1)
    x = x.flatten()
    y = test_df.corrected_pitches.to_numpy()
    
    # Model
    with pm.Model() as model:
        # Define priors
        a = pm.Normal('slope', 0, 16)
        b = pm.Normal('intercept', 0, 16)
        s = pm.Exponential('error', 1)

        # Predictions
        obs = pm.Normal('observation', a * x + b, s, observed = y)

        # Use MCMC to sample
        trace = pm.sample(1000, tune=1000, cores=1)

        # Get summary statistics for posterior
        posteriors = az.summary(trace, round_to = 2, hdi_prob = 0.95)
        agg_bayes[key] = posteriors.loc[["slope", "intercept", "error"], 
                                            ["mean", "sd", "hdi_2.5%", "hdi_97.5%"]]

# Need a better way of storing this data
# regular data frame with

Fitting linear model for: ('PU', 4)


<ipython-input-11-c1ad03f331f3>:30: FutureWarning: In v4.0, pm.sample will return an `arviz.InferenceData` object instead of a `MultiTrace` by default. You can pass return_inferencedata=True or return_inferencedata=False to be safe and silence this warning.
  trace = pm.sample(1000, tune=1000, cores=1)
Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Sequential sampling (2 chains in 1 job)
NUTS: [error, intercept, slope]


Sampling 2 chains for 1_000 tune and 1_000 draw iterations (2_000 + 2_000 draws total) took 6 seconds.
The acceptance probability does not match the target. It is 0.8908166762155325, but should be close to 0.8. Try to increase the number of tuning steps.


Fitting linear model for: ('PU', 5)


/opt/anaconda3/envs/fa/lib/python3.9/site-packages/pymc3/model.py:1755: ImputationWarning: Data in observation contains missing values and will be automatically imputed from the sampling distribution.
  warnings.warn(impute_message, ImputationWarning)
<ipython-input-11-c1ad03f331f3>:30: FutureWarning: In v4.0, pm.sample will return an `arviz.InferenceData` object instead of a `MultiTrace` by default. You can pass return_inferencedata=True or return_inferencedata=False to be safe and silence this warning.
  trace = pm.sample(1000, tune=1000, cores=1)
Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Sequential sampling (2 chains in 1 job)
NUTS: [observation_missing, error, intercept, slope]


...

<ipython-input-11-c1ad03f331f3>:30: FutureWarning: In v4.0, pm.sample will return an `arviz.InferenceData` object instead of a `MultiTrace` by default. You can pass return_inferencedata=True or return_inferencedata=False to be safe and silence this warning.
  trace = pm.sample(1000, tune=1000, cores=1)
Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Sequential sampling (2 chains in 1 job)
NUTS: [error, intercept, slope]


Sampling 2 chains for 1_000 tune and 1_000 draw iterations (2_000 + 2_000 draws total) took 5 seconds.
The acceptance probability does not match the target. It is 0.8886410480450131, but should be close to 0.8. Try to increase the number of tuning steps.


Fitting linear model for: ('FIH', 7)


/opt/anaconda3/envs/fa/lib/python3.9/site-packages/pymc3/model.py:1755: ImputationWarning: Data in observation contains missing values and will be automatically imputed from the sampling distribution.
  warnings.warn(impute_message, ImputationWarning)
<ipython-input-11-c1ad03f331f3>:30: FutureWarning: In v4.0, pm.sample will return an `arviz.InferenceData` object instead of a `MultiTrace` by default. You can pass return_inferencedata=True or return_inferencedata=False to be safe and silence this warning.
  trace = pm.sample(1000, tune=1000, cores=1)
Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Sequential sampling (2 chains in 1 job)
NUTS: [observation_missing, error, intercept, slope]


Sampling 2 chains for 1_000 tune and 1_000 draw iterations (2_000 + 2_000 draws total) took 8 seconds.
The acceptance probability does not match the target. It is 0.8866682736000073, but should be close to 0.8. Try to increase the number of tuning steps.
